# Serialization example

This example illustrates how to load a dataset from JSON, run the model on that dataset and write the output back to JSON. At the end the example is also shown for `msgpack`.

## Load dependencies

In [1]:
import json
import pprint
from pandas import DataFrame

from power_grid_model import PowerGridModel
from power_grid_model.utils import json_deserialize, json_serialize

## Load a dataset from a JSON file

The data is in the `data/serialized-input.json` file.

### Load the JSON file

In [2]:
with open("data/serialized_input.json") as fp:
    data = fp.read()

pprint.pprint(json.loads(data))

{'attributes': {'node': ['id', 'u_rated'],
                'source': ['id', 'node', 'status', 'u_ref', 'sk'],
                'sym_load': ['id',
                             'node',
                             'status',
                             'type',
                             'p_specified',
                             'q_specified']},
 'data': {'line': [{'c1': 4e-05,
                    'from_node': 1,
                    'from_status': 1,
                    'i_n': 500.0,
                    'id': 4,
                    'r1': 0.11,
                    'tan1': 0.1,
                    'to_node': 2,
                    'to_status': 1,
                    'x1': 0.12},
                   {'c1': 5e-05,
                    'from_node': 2,
                    'from_status': 1,
                    'i_n': 550.0,
                    'id': 5,
                    'r1': 0.15,
                    'tan1': 0.12,
                    'to_node': 3,
                    'to_status': 1,
        

### Deserialize the JSON data

In [3]:
dataset = json_deserialize(data)

print("components:", dataset.keys())
display(DataFrame(dataset["node"]))

components: dict_keys(['line', 'node', 'source', 'sym_load'])


,id,u_rated
0,1,10500.0
1,2,10500.0
2,3,10500.0


## Run power flow calculation on the loaded input data

In [4]:
model = PowerGridModel(dataset)
output = model.calculate_power_flow()

display(DataFrame(output["node"]))

,id,energized,u_pu,u,u_angle,p,q
0,1,1,1.030000,10815.000000,-2.527860e-14,2.408998e+06,-2.863495e+06
1,2,1,1.029997,10814.968183,-4.398000e-03,-1.010000e+06,-2.100000e+05
2,3,1,1.029484,10809.581008,-6.839956e-03,-1.020000e+06,-2.200000e+05


## Serialize the output dataset

### Default format

By default, the data is formatted nicely

In [5]:
serialized_output = json_serialize(output)

print(serialized_output)

{
  "attributes": {},
  "data": {
    "line": [
      {
        "energized": 1,
        "i_from": 199.76595459684984,
        "i_to": 141.69843328389055,
        "id": 4,
        "loading": 0.3995319091936997,
        "p_from": 2408997.8394389837,
        "p_to": -2252625.764367662,
        "q_from": -2863495.3646739325,
        "q_to": 1403928.5369474716,
        "s_from": 3742041.7279782956,
        "s_to": 2654305.591138379
      },
      {
        "energized": 1,
        "i_from": 108.73760885970873,
        "i_to": 55.731992269807684,
        "id": 5,
        "loading": 0.1977047433812886,
        "p_from": 1242625.7643675157,
        "p_to": -1019999.9999999403,
        "q_from": -1613928.5369477118,
        "q_to": -220000.00000002797,
        "s_from": 2036880.9765531549,
        "s_to": 1043455.7968595941
      }
    ],
    "node": [
      {
        "energized": 1,
        "id": 1,
        "p": 2408997.8394389837,
        "q": -2863495.3646739325,
        "u": 10815.0000000107

### Compact serialization

In the full result, all attributes are explicitly listed for each component.
In addition, all attributes are listed on a separate whiteline.
This is fairly expensive memory-wise.

If you need a more memory-efficient output, you can tell the serializer to use compact lists and to avoid using redundant newlines and whitespaces.

In [6]:
serialized_output = json_serialize(output, use_compact_list=True, indent=-1)

print(serialized_output)

{"attributes":{"line":["id","energized","loading","p_from","q_from","i_from","s_from","p_to","q_to","i_to","s_to"],"node":["id","energized","u_pu","u","u_angle","p","q"],"source":["id","energized","p","q","i","s","pf"],"sym_load":["id","energized","p","q","i","s","pf"]},"data":{"line":[[4,1,0.3995319091936997,2408997.8394389837,-2863495.3646739325,199.76595459684984,3742041.7279782956,-2252625.764367662,1403928.5369474716,141.69843328389055,2654305.591138379],[5,1,0.1977047433812886,1242625.7643675157,-1613928.5369477118,108.73760885970873,2036880.9765531549,-1019999.9999999403,-220000.00000002797,55.731992269807684,1043455.7968595941]],"node":[[1,1,1.0300000000010254,10815.000000010767,-2.5278595360047667e-14,2408997.8394389837,-2863495.3646739325],[2,1,1.029996969815605,10814.968183063853,-0.004397999804754551,-1010000.0000001461,-210000.0000001819],[3,1,1.029483905569344,10809.581008478113,-0.006839956175380042,-1019999.9999999403,-220000.00000002797]],"source":[[15,1,-7840728.15070

The compact result is still valid JSON

In [7]:
pprint.pprint(json.loads(serialized_output))

{'attributes': {'line': ['id',
                         'energized',
                         'loading',
                         'p_from',
                         'q_from',
                         'i_from',
                         's_from',
                         'p_to',
                         'q_to',
                         'i_to',
                         's_to'],
                'node': ['id', 'energized', 'u_pu', 'u', 'u_angle', 'p', 'q'],
                'source': ['id', 'energized', 'p', 'q', 'i', 's', 'pf'],
                'sym_load': ['id', 'energized', 'p', 'q', 'i', 's', 'pf']},
 'data': {'line': [[4,
                    1,
                    0.3995319091936997,
                    2408997.8394389837,
                    -2863495.3646739325,
                    199.76595459684984,
                    3742041.7279782956,
                    -2252625.764367662,
                    1403928.5369474716,
                    141.69843328389055,
                    2654305

## Msgpack serialization

To have even higher performance and smaller space, you can use the binary format [`msgpack`](https://msgpack.org/). The example below shows a round trip to dump and load `msgpack` data, and intantiate model.

### Serialize to msgpack

We can serialize the output data into `msgpack` binary with and without compact list. The result is a `bytes` object. See the resulted differences in length of the data. The differences will be significant when you have a large dataset.

In [8]:
from power_grid_model.utils import msgpack_serialize

msgpack_data_not_compact = msgpack_serialize(output, use_compact_list=False)
msgpack_data_compact = msgpack_serialize(output, use_compact_list=True)

print(f"Type of the returned objects: {type(msgpack_data_not_compact)}, {type(msgpack_data_compact)}")
print(f"Length of not-compact data: {len(msgpack_data_not_compact)}")
print(f"Length of compact data: {len(msgpack_data_compact)}")

Type of the returned objects: <class 'bytes'>, <class 'bytes'>
Length of not-compact data: 977
Length of compact data: 802


### Deserialize from msgpack

We can deserialize the data we just created. We then print the node result. Note that they are exactly the same.

In [9]:
from power_grid_model.utils import msgpack_deserialize

output_data_not_compact = msgpack_deserialize(msgpack_data_not_compact)
output_data_compact = msgpack_deserialize(msgpack_data_compact)

print("----Node result from not compact data----")
print(DataFrame(output_data_not_compact["node"]))
print("----Node result from compact data----")
print(DataFrame(output_data_compact["node"]))

----Node result from not compact data----
   id  energized      u_pu             u       u_angle             p  \
0   1          1  1.030000  10815.000000 -2.527860e-14  2.408998e+06   
1   2          1  1.029997  10814.968183 -4.398000e-03 -1.010000e+06   
2   3          1  1.029484  10809.581008 -6.839956e-03 -1.020000e+06   

              q  
0 -2.863495e+06  
1 -2.100000e+05  
2 -2.200000e+05  
----Node result from compact data----
   id  energized      u_pu             u       u_angle             p  \
0   1          1  1.030000  10815.000000 -2.527860e-14  2.408998e+06   
1   2          1  1.029997  10814.968183 -4.398000e-03 -1.010000e+06   
2   3          1  1.029484  10809.581008 -6.839956e-03 -1.020000e+06   

              q  
0 -2.863495e+06  
1 -2.100000e+05  
2 -2.200000e+05  
